In [10]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore')

In [11]:
#Importing libraries and Reading the Data

In [12]:
df = pd.read_csv("StudentsPerformance.csv")

In [13]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [15]:
# Data wrangling

In [16]:
df.isna().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [17]:
df.duplicated().sum()

0

In [18]:
df=data.rename({'test preparation course':'test_preparation_course','reading score':'reading_score',\
                  'parental level of education':'parental_levelof_education','math score':'math_score',\
                 'writing score':'writing_score',"race/ethnicity":'race'},axis=1)

NameError: name 'data' is not defined

In [19]:
df.test_preparation_course.unique()

AttributeError: 'DataFrame' object has no attribute 'test_preparation_course'

In [ ]:
#Finding correlations between data atributes

In [ ]:
df_corr = data.iloc[:,4:]

In [ ]:
df_corr.test_preparation_course=data_corr.test_preparation_course.replace({'none':0, 'completed':1})

In [ ]:
df_corr.head()

In [ ]:
df_corr=data_corr.corr()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df=data_corr,annot=True)

In [ ]:
plt.figure(figsize=(30,28))
for i, col in enumerate( ['math_score', 'reading_score','writing_score']):
    plt.subplot(3, 3, i+1)
    sns.histplot(data = data,
            x = col,
            kde = True,
            bins = 30,
            color = 'blue')

plt.show()

In [ ]:
plt.figure(figsize=(12,9))
for i,col in enumerate(['parental_levelof_education','gender', 'race', 'lunch','test_preparation_course']):
    plt.subplot(3,2,i+1)
    x=data[col].value_counts().reset_index()
    plt.title(col)
    plt.pie(x=x['count'],labels=x[col],autopct="%0.1f%%",colors=sns.color_palette('muted'))

In [ ]:
data.describe()


In [ ]:
#Multivariate analysis and asking questions about the data

In [ ]:
sns.scatterplot(data=data,x="writing_score",y="reading_score",hue="test_preparation_course",palette=sns.color_palette('Dark2'))
plt.show()

In [ ]:
sns.scatterplot(data=data,y="math_score",x="writing_score",hue="test_preparation_course",palette=sns.color_palette('Dark2'))
plt.show()

In [ ]:
#Does gender,race or parental level of education have an effect on scores?

In [ ]:
data.groupby("gender")[["reading_score","writing_score","math_score"]].mean().plot(kind="bar",)
plt.legend(bbox_to_anchor=(1.4,0.8))
plt.title("Average score of each subject across the two genders")
plt.show()

In [ ]:
#it can be seen that females have higher average scores in reading and writing while males are higher in math

In [ ]:
data.groupby("race")[["reading_score","writing_score","math_score"]].mean().plot(kind="bar",)
plt.legend(bbox_to_anchor=(1.4,0.8))
plt.title("Average score of each subject across race groups")
plt.show()

In [ ]:
#As seen group E has the highest average scores followed by D,C,B and finally A

In [ ]:
data.groupby("parental_levelof_education")[["reading_score","writing_score","math_score"]].mean().plot(kind="bar",)
plt.legend(bbox_to_anchor=(1.4,0.8))
plt.title("Average score of each subject across the parent's level of education")
plt.show()

In [ ]:
data.groupby(['lunch','parental_levelof_education'])["lunch"].count()

In [ ]:
#Does race or parental level of education affect the lunch taken by students?

In [ ]:
plt.figure(figsize=(12,9))
sns.countplot(data=data,x='parental_levelof_education',hue='lunch',palette=sns.color_palette('hls'))
plt.show()

In [ ]:
plt.figure(figsize=(12,9))
sns.countplot(data=data,x='race',hue='lunch',palette=sns.color_palette('hls'))
plt.show()

In [ ]:
#Both race and parent's level of education seem to have no effect on lunch taken by students